# Re-grid mirrored CMIP6 data on ACDN

Use this notebook to re-grid all of the mirrored CMIP6 data to a common grid. We will use the grid that is common between CESM2, CESM2-WACCM, NorESM2, and TaiESM1.

Instead of just re-gridding all files, we will take a manifest specific approach, and re-grid by model, frequency, variable, etc. I think these regridded files should be saved in their own folder, since it is sort of a derived product? I.e., "CMIP6 outputs on a common grid" feels like its own thing. The alternative would be to save regridded files in this native ESGF directory structure, but I worry that could cause confusion because then we have derived data mixed in with mirrored data. It seems like it might be nice if the `/beegfs/CMIP6/arctic-cmip6/CMIP6` directory excluded everything but mirrored data. So, we will make another directory for the regridded data: `/beegfs/CMIP6/arctic-cmip6/CMIP6_common`

Test re-gridding historical monthly data from one grid to another

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe

from config import *
from slurm import *

# ignore serializationWarnings from xarray for datasets with multiple FillValues
import warnings
warnings.filterwarnings("ignore", category=xr.SerializationWarning)

Iterate over the batch files and create slurm `sbatch` scripts to regrid all filepaths contained in those files (one job for each file).

In [3]:
sbatch_fps = []
sbatch_dir = slurm_dir.joinpath("regrid")
sbatch_dir.mkdir(exist_ok=True)
for fp in regrid_batch_dir.glob("*.txt"):
    sbatch_str = fp.name.split("batch_")[1].split(".txt")[0]
    sbatch_fp = sbatch_dir.joinpath(f"regrid_{sbatch_str}.slurm")
    # filepath for slurm stdout
    sbatch_out_fp = sbatch_dir.joinpath(sbatch_fp.name.replace(".slurm", "_%j.out"))
    sbatch_head = make_sbatch_head(**sbatch_head_kwargs)
    sbatch_regrid_kwargs = {
        "sbatch_fp": sbatch_fp,
        "sbatch_out_fp": sbatch_out_fp,
        "regrid_script": regrid_script,
        "regrid_dir": regrid_dir,
        "regrid_batch_fp": fp,
        "dst_fp": target_grid_fp,
        "sbatch_head": sbatch_head,
    }
    write_sbatch_regrid(**sbatch_regrid_kwargs)
    sbatch_fps.append(sbatch_fp)

Remove existing slurm output files if desired:

In [4]:
_ = [fp.unlink() for fp in sbatch_dir.glob("*.out")]

Submit the jobs:

In [9]:
job_ids = [submit_sbatch(fp) for fp in sbatch_fps]

End.